In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
from model import get_model
from torchvision import transforms
from utils.output_utils import prepare_output
from PIL import Image
import time
from argparse import Namespace

In [2]:
torch.__version__

'1.0.1.post2'

In [22]:
data_dir = '../data'
use_gpu = True
device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
map_loc = None if torch.cuda.is_available() and use_gpu else 'cpu'

In [23]:
# code below was used to save vocab files so that they can be loaded without Vocabulary class
#ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_ingrs.pkl'), 'rb'))
#ingrs_vocab = [min(w, key=len) if not isinstance(w, str) else w for w in ingrs_vocab.idx2word.values()]
#vocab = pickle.load(open(os.path.join(data_dir, 'final_recipe1m_vocab_toks.pkl'), 'rb')).idx2word
#pickle.dump(ingrs_vocab, open('../demo/ingr_vocab.pkl', 'wb'))
#pickle.dump(vocab, open('../demo/instr_vocab.pkl', 'wb'))

ingrs_vocab = pickle.load(open(os.path.join(data_dir, 'ingr_vocab.pkl'), 'rb'))
vocab = pickle.load(open(os.path.join(data_dir, 'instr_vocab.pkl'), 'rb'))

ingr_vocab_size = len(ingrs_vocab)
instrs_vocab_size = len(vocab)
output_dim = instrs_vocab_size

In [4]:
ingrs_vocab

['<end>',
 'macaroni',
 'cheese',
 'celery',
 'pimentos',
 'mayonnaise',
 'vinegar',
 'salt',
 'dill',
 'onion',
 'pepper',
 'cucumber',
 'oil',
 'basil',
 'gelatin',
 'water',
 'cool_whip',
 'watermelon',
 'cracker',
 'coconut',
 'beef',
 'garlic',
 'juice',
 'soy_sauce',
 'cornstarch',
 'pineapple',
 'nuts',
 'chicken',
 'tea',
 'rhubarb',
 'sugar',
 'strawberry',
 'cake',
 'butter',
 'flour',
 'cinnamon',
 'baking_soda',
 'baking_powder',
 'vanilla',
 'zucchini',
 'walnuts',
 'ginger',
 'rice',
 'greens',
 'fat',
 'banana',
 'kiwi',
 'yogurt',
 'seeds',
 'tenderloin',
 'fennel',
 'wine',
 'broth',
 'brandy',
 'liqueur',
 'orange',
 'lemon',
 'sprite',
 'ice',
 'egg',
 'pecans',
 'medium_cheddar',
 'hamburger',
 'oats',
 'chili',
 'milk',
 'corn',
 'broccoli',
 'oregano',
 'extract',
 'teriyaki_sauce',
 'cream',
 'cookie',
 'stevia',
 'wafers',
 'lemonade',
 'chips',
 'bread',
 'half_-_and_-_half',
 'syrup',
 'crabmeat',
 'bacon',
 'cilantro',
 'seasoning',
 'sausage',
 'spaghetti',


In [24]:
# Hard coded args for the model.
args = Namespace(aux_data_dir='../data', batch_size=128, beam=-1, crop_size=224, decay_lr=True, dropout_decoder_i=0.3, dropout_decoder_r=0.3, dropout_encoder=0.3, embed_size=512, es_metric='loss', eval_split='val', finetune_after=-1, get_perplexity=False, greedy=False, image_model='resnet50', image_size=256, ingrs_only=True, label_smoothing_ingr=0.1, learning_rate=0.001, log_step=10, log_term=False, loss_weight=[1.0, 0.0, 0.0, 0.0], lr_decay_every=1, lr_decay_rate=0.99, max_eval=4096, maxnumims=5, maxnuminstrs=10, maxnumlabels=20, maxseqlen=15, model_name='model', n_att=8, n_att_ingrs=4, num_epochs=400, num_workers=8, numgens=3, patience=50, project_name='inversecooking', recipe1m_dir='path/to/recipe1m', recipe_only=False, resume=False, save_dir='path/to/save/models', scale_learning_rate_cnn=0.01, suff='', temperature=1.0, tensorboard=True, transf_layers=16, transf_layers_ingrs=4, transfer_from='', use_lmdb=True, use_true_ingrs=False, weight_decay=0.0)

In [25]:
t = time.time()
import sys; sys.argv=['']; del sys
args.maxseqlen = 15
args.ingrs_only=True
model = get_model(args, ingr_vocab_size, instrs_vocab_size)
# Load the trained model parameters
model_path = os.path.join(data_dir, 'modelbest.ckpt')
model.load_state_dict(torch.load(model_path, map_location=map_loc))
model.to(device)
model.eval()
model.ingrs_only = True
print ('loaded model')
print ("Elapsed time:", time.time() -t)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


loaded model
Elapsed time: 2.6416473388671875


In [9]:
ingredient_encoder = model.ingredient_encoder

In [8]:
from torch import LongTensor

In [37]:
ingredient_encoder.linear.weight

Parameter containing:
tensor([[-0.1117, -0.4966,  0.1631,  ...,  0.9230, -0.6661, -0.6180],
        [ 1.3278,  1.6117, -0.1660,  ..., -1.6526, -0.8649, -0.2160],
        [ 0.1153,  0.3926,  0.1415,  ..., -0.1767, -0.1499,  0.9249],
        ...,
        [-0.6577,  0.2762, -0.5360,  ..., -0.7316,  1.7329,  2.2061],
        [ 1.1965,  0.1213,  0.4309,  ..., -0.2971,  0.7490, -0.1985],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', requires_grad=True)

In [46]:
ingr_vocab_size

1488

In [10]:
input_first = torch.cat((LongTensor([0, 1]), LongTensor(range(1486))))

In [11]:
LongTensor(0).to(device)

tensor([], device='cuda:0', dtype=torch.int64)

In [12]:
input_first = input_first.to(device)

tensor([   0,    1,    0,  ..., 1483, 1484, 1485], device='cuda:0')

In [19]:
from model import label2onehot
from torch import Tensor

In [13]:
ingredient_encoder(input_first, onehot_flag=True)

RuntimeError: Expected object of backend CPU but got backend CUDA for argument #2 'mat2'